## Описание проекта

### Бизнес задача
Проанализировать базу данных сервис для чтения книг по подписке. В ней информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. 
Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных
Таблица books. Содержит данные о книгах:
* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.
Таблица authors. Содержит данные об авторах:
* author_id — идентификатор автора;
* author — имя автора.
Таблица publishers. Содержит данные об издательствах:
* publisher_id — идентификатор издательства;
* publisher — название издательства;
Таблица ratings. Содержит данные о пользовательских оценках книг:
* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.
Таблица reviews. Содержит данные о пользовательских обзорах на книги:
* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя пользователя, написавшего обзор;
* text — текст обзора.
### Задание
* Посчитать, сколько книг вышло после 1 января 2000 года;
* Для каждой книги посчитать количество обзоров и среднюю оценку;
* Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
* Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
* Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Подготовка к выполнению заданий

### Подготовка рабочей среды и функций

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
# устанавливаем параметры

db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Создадим функцию для вывода результата 

In [3]:
def select(sql):
    return pd.read_sql(sql,con = engine)

Добавим функцию для вывода основной информации о будущих таблицах

In [4]:
def total_info_data(data):
    ''' Данная функция предобработки делает следующие вещи:
        1) Проверяет названия столбцов, приводит их к стилю pep8
        2) Удаляет явные дубликаты
        3) Выводит информацию методом info()
        4) Выводит информацию методом describe()
        5) Выводит информацию о пропусках 
        6) Приводит столбцы с датой к правильному типу'''
       
    
    col = []
    for i in data.columns:
        if bool(re.search('([A-Z])', i))==True:
            col.append(i.replace(f'{list(re.findall("[A-Z]+",i))[0]}',f'_{list(re.findall("[A-Z]+",i))[0].lower()}'))
        else:
            col.append(i)
    
    for j in range(len(col)):
        if col[j][0]=='_':
            col[j]=col[j].replace(f'{col[j]}',f'{col[j][1:]}')
            
    data.columns = col
    del col
    
    for i in data.columns:
        if 'dt' in i or 'date' in i:
            data[i] = pd.to_datetime(data[i], yearfirst=True)
    # Приводим колонки к нижнему регистру и заменяем
    # пробелы на _ для приведения к корректному стилю написания
    data.columns = data.columns.str.lower().str.replace(' ', '_').str.replace('.','_')
    dup_sum = data.duplicated().sum()
    # Удаляем явные дубликаты
    data = data.drop_duplicates().reset_index(drop=True) 
    # Выводим 30 случайных строк датафрейма
    random_str = display(data.sample(frac=0.25).head(10))
    # Получаем общую информацию о датасете
    info = data.info()
    # Получаем информарцию о пропусках
    nan = data.isna().sum()
    # Получаем полное описание датафрейма
    describe = display(data.describe(include = 'all'))  # Получаем полное описание датафрейма

    return random_str, info, describe, nan

### Получение основных таблиц

Выведем данные из таблиц "books", "authors", "publishers", "ratings" и "reviews"

###### Посмотрим данные из таблицы books

In [5]:
books = '''
SELECT * FROM books
'''

In [6]:
total_info_data(select(books))

,book_id,author_id,title,num_pages,publication_date,publisher_id
477,478,291,Nineteen Minutes,440,2007-03-05,27
204,205,255,Dubliners,207,2001-03-15,203
132,133,536,Can You Keep a Secret?,374,2005-12-27,82
295,296,271,Hard Eight (Stephanie Plum #8),326,2003-06-16,284
871,872,277,The Social Contract,168,2006-05-30,210
923,924,178,This Present Darkness (Darkness #1),376,2003-06-26,67
935,936,507,Treasure Island,311,2001-09-15,166
158,159,207,Confessions of an Ugly Stepsister,372,2000-10-03,333
729,730,69,The Four Loves,170,2002-06-05,136
811,812,591,The Novice (Black Magician Trilogy #2),577,2004-07-01,198


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   book_id           1000 non-null   int64         
 1   author_id         1000 non-null   int64         
 2   title             1000 non-null   object        
 3   num_pages         1000 non-null   int64         
 4   publication_date  1000 non-null   datetime64[ns]
 5   publisher_id      1000 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 47.0+ KB


,book_id,author_id,title,num_pages,publication_date,publisher_id
count,1000.000000,1000.000000,1000,1000.00000,1000,1000.000000
unique,NaN,NaN,999,NaN,618,NaN
top,NaN,NaN,Memoirs of a Geisha,NaN,2006-08-29 00:00:00,NaN
freq,NaN,NaN,2,NaN,10,NaN
first,NaN,NaN,NaN,NaN,1952-12-01 00:00:00,NaN
last,NaN,NaN,NaN,NaN,2020-03-31 00:00:00,NaN
mean,500.500000,320.417000,NaN,389.11100,NaN,171.270000
std,288.819436,181.620172,NaN,229.39014,NaN,99.082685
min,1.000000,1.000000,NaN,14.00000,NaN,1.000000
25%,250.750000,162.750000,NaN,249.00000,NaN,83.000000


(None,
 None,
 None,
 book_id             0
 author_id           0
 title               0
 num_pages           0
 publication_date    0
 publisher_id        0
 dtype: int64)

* В таблице books 1000 строк и 6 столбцов
* Отсутствуют пропуски и дубликаты
* Информация о книгах в нашей базе с 1952 года по 2020
* 999 книг уникальных
* 75% книг ограничиваются 453 страницами, есть аномально большие книги на 2690 страниц

###### Посмотрим данные из таблицы authors

In [7]:
authors = '''
SELECT * FROM authors
'''

In [8]:
total_info_data(select(authors))

,author_id,author
409,410,Megan Whalen Turner
290,291,Jodi Picoult
589,590,Tracy Kidder
477,478,Philip K. Dick/David Alabort/Manuel Espín
172,173,Evelyn Waugh
193,194,George Eliot/A.S. Byatt
314,315,Jon Stone/Michael J. Smollin
588,589,Toni Morrison
496,497,Richard Wright
523,524,Salman Rushdie


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


,author_id,author
count,636.000000,636
unique,NaN,636
top,NaN,Rachel Cohn/David Levithan
freq,NaN,1
mean,318.500000,NaN
std,183.741666,NaN
min,1.000000,NaN
25%,159.750000,NaN
50%,318.500000,NaN
75%,477.250000,NaN


(None,
 None,
 None,
 author_id    0
 author       0
 dtype: int64)

* В таблице authors 636 строк и 2 столбца 
* Отсутствуют пропуски и дубликаты

###### Посмотрим данные из таблицы publishers

In [9]:
publishers = '''
SELECT * FROM publishers
'''

In [10]:
total_info_data(select(publishers))

,publisher_id,publisher
279,280,Spectra Books
148,149,Harvest Books
129,130,Harper Perennial Modern Classics
238,239,Random House Audio Publishing Group
76,77,Del Rey Books
16,17,Anchor Books/Knopf Doubleday Publishing Group
18,19,Applewood Books
315,316,Virago
167,168,Knopf Books for Young Readers
246,247,Riverhead Books


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


,publisher_id,publisher
count,340.000000,340
unique,NaN,340
top,NaN,Bantam Doubleday Dell (NYC)
freq,NaN,1
mean,170.500000,NaN
std,98.293777,NaN
min,1.000000,NaN
25%,85.750000,NaN
50%,170.500000,NaN
75%,255.250000,NaN


(None,
 None,
 None,
 publisher_id    0
 publisher       0
 dtype: int64)

* В таблице publishers 340 строк и 2 столбца
* Отсутствуют пропуски и дубликаты

###### Посмотрим данные из таблицы ratings

In [11]:
ratings = '''
SELECT * FROM ratings
'''

In [12]:
total_info_data(select(ratings))

,rating_id,book_id,username,rating
4317,4318,693,tnolan,3
2901,2902,455,robert57,5
3809,3810,624,wknight,5
2750,2751,426,adamsabigail,4
154,155,33,mariokeller,3
1930,1931,302,amymckenzie,5
3904,3905,631,danielchung,3
2828,2829,437,jessica49,3
6145,6146,948,conradanita,3
645,646,98,masonsara,5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


,rating_id,book_id,username,rating
count,6456.000000,6456.000000,6456,6456.000000
unique,NaN,NaN,160,NaN
top,NaN,NaN,martinadam,NaN
freq,NaN,NaN,56,NaN
mean,3228.500000,510.574195,NaN,3.928284
std,1863.831001,284.141636,NaN,0.943303
min,1.000000,1.000000,NaN,1.000000
25%,1614.750000,291.000000,NaN,3.000000
50%,3228.500000,506.000000,NaN,4.000000
75%,4842.250000,750.000000,NaN,5.000000


(None,
 None,
 None,
 rating_id    0
 book_id      0
 username     0
 rating       0
 dtype: int64)

* В таблице ratings 6456 строк и 4 столбца 
* Отсутствуют пропуски и дубликаты
* Всего 160 уникальных пользователя сделали 6456 оценок, самый часто встречающийся юзер сделал 56 оценок
* Значения рейтинга лежат в диапазоне от 1 до 5, все корректно

###### Посмотрим данные из таблицы reviews

In [14]:
total_info_data(select(reviews))

,review_id,book_id,username,text
850,851,311,tnovak,Pretty throughout size serve grow company. Abo...
196,196,68,webbmichelle,Soon travel get play control me pressure. Unde...
2683,2684,960,nthomas,Note to certainly third entire budget. True po...
1475,1475,536,timothygibson,Lead simple under local. Economy war discussio...
775,774,287,rbender,How record air high what worker. Debate bring ...
1955,1954,705,shermannatalie,Wait rise machine letter table six only. Try t...
2004,2003,724,fconrad,Very heavy morning standard democratic cut.
2439,2439,873,michael57,Once reach carry nor result budget. Cost estab...
1588,1587,577,briannachavez,Political son draw else stay special third. So...
1239,1239,452,edwardbrown,Room study rather fast organization item accor...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


,review_id,book_id,username,text
count,2793.000000,2793.000000,2793,2793
unique,NaN,NaN,160,2793
top,NaN,NaN,susan85,Experience one around start Mrs other test. No...
freq,NaN,NaN,29,1
mean,1397.000000,504.693161,NaN,NaN
std,806.413976,288.472931,NaN,NaN
min,1.000000,1.000000,NaN,NaN
25%,699.000000,259.000000,NaN,NaN
50%,1397.000000,505.000000,NaN,NaN
75%,2095.000000,753.000000,NaN,NaN


(None,
 None,
 None,
 review_id    0
 book_id      0
 username     0
 text         0
 dtype: int64)

* В таблице reviews 2793 строк и 4 столбца
* Отсутствуют пропуски и дубликаты
* 160 уникальных ользователей сделали 2793 ревью, самый часто встречающийся юзер сделал 29 ревью

### Вывод
* Все таблицы без прпоусков и дубликатов
* Имеют корректный тип данных
* Аномальных значений почти не замечено, только объем страниц вызвал сомнения, но для задания нам это не важно

## Выполнение заданий

### Задание 1

###### Посчитаем сколько книг, выпущено после 1 января 2000 года

In [15]:
books_count = '''
SELECT COUNT(book_id) FROM books WHERE publication_date > '2000-01-01'
'''

In [16]:
select(books_count)

,count
0,819


После 1 января 2000 года было выпущено 819 книг

### Задание 2

###### Посчитаем для каждой книги количество обзоров и среднюю оценку

In [17]:
book_reviews = '''
SELECT books.title,
    books.book_id,
    AVG(ratings.rating) AS avg,
    COUNT(DISTINCT reviews.text) AS cnt
FROM
    books
LEFT JOIN ratings ON ratings.book_id = books.book_id
LEFT JOIN reviews ON reviews.book_id = books.book_id
GROUP BY
    books.title,
    books.book_id
ORDER BY
    cnt DESC

'''

In [18]:
select(book_reviews)

,title,book_id,avg,cnt
0,Twilight (Twilight #1),948,3.662500,7
1,Water for Elephants,963,3.977273,6
2,The Glass Castle,734,4.206897,6
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,4.414634,6
4,The Curious Incident of the Dog in the Night-Time,695,4.081081,6
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,3.666667,0
996,The Natural Way to Draw,808,3.000000,0
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,5.000000,0
998,Essential Tales and Poems,221,4.000000,0


In [19]:
select(book_reviews).describe()

,book_id,avg,cnt
count,1000.000000,1000.000000,1000.000000
mean,500.500000,3.898973,2.793000
std,288.819436,0.562376,1.074852
min,1.000000,1.500000,0.000000
25%,250.750000,3.500000,2.000000
50%,500.500000,4.000000,3.000000
75%,750.250000,4.333333,3.000000
max,1000.000000,5.000000,7.000000


* Максимальное кол-во обзоров у книги Twilight (Twilight #1) - 7, затем следует множество книг с количеством обзоров - 6,
75% книг имеет 2 или больше оценки
* 50% книг имеют оценку в диапазоне 3.5 - 4.3.
* Есть ряд книг без обзоров но с оценкой

### Задание 3

###### Определим издательство, которое издало наибольшее число книг толще 50 страниц

In [20]:
book_publisher = '''
SELECT
    publishers.publisher AS name,
    COUNT(books.book_id) AS cnt
FROM
    publishers
INNER JOIN books ON books.publisher_id = publishers.publisher_id
WHERE
    books.num_pages > 50
GROUP BY
    name
ORDER BY 
    cnt DESC

'''

In [21]:
select(book_publisher).head()

,name,cnt
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19


In [22]:
select(book_publisher).describe()

,cnt
count,334.000000
mean,2.970060
std,4.377015
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,42.000000


* Издательство Penguin Books - выпустило наибольше число книг в кол-ве 42 книг, толще 50 страниц, затем следуют издательства Vintage (31) и Grand Central Publishing (25)
* Как миниму 50% издательств выпустило всего по 1 книге больше 50 страниц. Около 25% издательств выпустило 3 и более книги толще 50 страниц.

### Задание 4

######  Определим автора с самой высокой средней оценкой книг (учитываем только книги с 50 и более оценками).

In [23]:
best_author = '''
SELECT
    ABAC.author AS author,
    AVG(ABAC.avg_rating) AS avg_rating
FROM
(SELECT
    authors.author AS author,
    books.book_id AS id,
    AVG(ratings.rating) AS avg_rating,
    COUNT(ratings.rating) AS cnt_rating
FROM
    authors
INNER JOIN books ON books.author_id = authors.author_id
INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY
    author,
    id
    HAVING
    COUNT(ratings.rating) >= 50) AS ABAC
GROUP BY
    author
ORDER BY
    avg_rating DESC
'''

In [24]:
select(best_author)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Cамая высокая средняя оценка у автора J.K. Rowling/Mary GrandPré (4.2838), далее Markus Zusak/Cao Xuân Việt Khương (4.264) и J.R.R. Tolkien (4.258)


### Задание 5

######  Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [25]:
avg_text = '''
SELECT ROUND(avg(count)) 
FROM
(SELECT
    COUNT(text)
FROM 
    reviews
FULL OUTER JOIN (SELECT username,
    COUNT(rating_id) as cnt
FROM 
    ratings
GROUP BY
    username) AS cnt_rating
ON cnt_rating.username = reviews.username
WHERE cnt > 50
GROUP BY reviews.username) as count
'''

In [26]:
select(avg_text)

,round
0,24.0


Среднее кол-во обзоров пользователей, которые поставили более чем по 50 оценок равно 24.

### Общий вывод:

**Итоговые наблюдения:**

Оценки и рейтинги:

* Максимальное кол-во обзоров у книги Twilight (Twilight #1) - 7, затем следует множество книг с количеством обзоров - 6,
75% книг имеет 2 или больше оценки


* 50% книг имеют оценку в диапазоне 3.5 - 4.3.

Издательства:

* Издательство Penguin Books - выпустило наибольше число книг в кол-ве 42 книг, толще 50 страниц, затем следуют издательства Vintage (31) и Grand Central Publishing (25)
* Как миниму 50% издательств выпустило всего по 1 книге больше 50 страниц. Около 25% издательств выпустило 3 и более книги толще 50 страниц.

Авторы:

Cамая высокая средняя оценка у автора J.K. Rowling/Mary GrandPré (4.2838), далее Markus Zusak/Cao Xuân Việt Khương (4.264) и J.R.R. Tolkien (4.258)


**Рекомендации:**

После короткого анализа, можно сделать ряд рекомендаций, в какую сторону смотреть для развития нашего продукта

Стоит обратить внимание на книги, которые:
* имеют 2 и более оценки
* авторы которых, имеют средний рейтинг больше 4

Для некоторых из этих книг, можно устроить промо, прослушать/прочитать книгу без подписки. Это поможет пользователям проявить больше лояльности, и осесть в нашем приложение, став платящим клиентом

Также стоит обратить внимание на самые крупные издательства, возможно с ними можно организовать сотруднечество для перевода/озвучивания книг